In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

/data/jiacheng/miniconda3/envs/common/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
# 步骤 1：找到排序最大的文件夹
output_path = "/home/ljc/data/graphrag/alltest/location_med_exp/dataset4_v2_exp2_ongo_five_v2/output/"
folders = [os.path.join(output_path, d) for d in os.listdir(output_path) if os.path.isdir(os.path.join(output_path, d))]
latest_folder = max(folders, key=os.path.getmtime)

In [4]:
INPUT_DIR = latest_folder + "/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [5]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 1191


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,TOKYO UNIVERSITY,ORGANIZATION,"Tokyo University, established in 1877, is one ...",b7f1364f580cb7a70758342a38c9da83,0,2,0,b45241d70f0e43fca764df95b2b81f77,2.0,"[-0.06433810293674469, 0.030881628394126892, -...",None,b45241d70f0e43fca764df95b2b81f77,10.319826,-3.012268
1,0,WASEDA UNIVERSITY,ORGANIZATION,"Waseda University, located in Tokyo, is renown...",b7f1364f580cb7a70758342a38c9da83,0,2,1,4119fd06010c494caa07f439b333f4c5,2.0,"[-0.06056540831923485, 0.026886945590376854, -...",None,4119fd06010c494caa07f439b333f4c5,10.240582,-3.500757
2,0,KEIO UNIVERSITY,ORGANIZATION,"Keio University, situated in Tokyo, is a priva...",b7f1364f580cb7a70758342a38c9da83,0,2,2,d3835bf3dda84ead99deadbeac5d0d7d,2.0,"[-0.05159485340118408, 0.022467393428087234, -...",None,d3835bf3dda84ead99deadbeac5d0d7d,11.141186,-4.997539
3,0,TOKYO INSTITUTE OF TECHNOLOGY,ORGANIZATION,"Tokyo Institute of Technology, often referred ...",b7f1364f580cb7a70758342a38c9da83,0,2,3,077d2820ae1845bcbb1803379a3d1eae,2.0,"[-0.05269402265548706, 0.01943577080965042, -0...",None,077d2820ae1845bcbb1803379a3d1eae,10.507893,-4.529240
4,0,SOPHIA UNIVERSITY,ORGANIZATION,Sophia University is a prestigious private uni...,"4a4fbb39e99ee454e8cb7ab44effac1b,b7f1364f580cb...",0,3,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,3.0,"[-0.032251447439193726, 0.013826754875481129, ...",None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,9.900887,2.819891


In [30]:
# 过滤出 title 为指定值的行
tokyo_university_df = entity_df[entity_df["title"].isin(["PEKING UNIVERSITY","BEIJING", "TOKYO"])]

# 显示结果
tokyo_university_df
 

,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
5,0,TOKYO,GEO,"Tokyo is the capital city of Japan, renowned f...","0c2c77fdb651261f582db1c092ce1f18,1ca3194384f65...",0,15,5,19a7f254a5d64566ab5cc15472df02de,15.0,"[-0.12152056396007538, 0.048213645815849304, -...",None,19a7f254a5d64566ab5cc15472df02de,11.244842,-2.451398
75,0,BEIJING,GEO,"Beijing is the capital city of China, known fo...","0c2c77fdb651261f582db1c092ce1f18,1ca3194384f65...",0,10,75,ed6d2eee9d7b4f5db466b1f6404d31cc,10.0,"[-0.07640577852725983, 0.03162377327680588, -0...",None,ed6d2eee9d7b4f5db466b1f6404d31cc,11.885282,-3.073426
252,0,PEKING UNIVERSITY,ORGANIZATION,Peking University is a public university locat...,cffba679617724f608225fbd119bda85,0,1,252,5f1fc373a8f34050a5f7dbd8ac852c1b,1.0,"[-0.049104541540145874, 0.01859055832028389, -...",None,5f1fc373a8f34050a5f7dbd8ac852c1b,11.512719,-4.340917
402,1,TOKYO,GEO,"Tokyo is the capital city of Japan, renowned f...","0c2c77fdb651261f582db1c092ce1f18,1ca3194384f65...",26,15,5,19a7f254a5d64566ab5cc15472df02de,15.0,"[-0.11892606317996979, 0.0008972971700131893, ...",None,19a7f254a5d64566ab5cc15472df02de,11.244842,-2.451398
472,1,BEIJING,GEO,"Beijing is the capital city of China, known fo...","0c2c77fdb651261f582db1c092ce1f18,1ca3194384f65...",25,10,75,ed6d2eee9d7b4f5db466b1f6404d31cc,10.0,"[-0.07926836609840393, -0.029859475791454315, ...",None,ed6d2eee9d7b4f5db466b1f6404d31cc,11.885282,-3.073426
649,1,PEKING UNIVERSITY,ORGANIZATION,Peking University is a public university locat...,cffba679617724f608225fbd119bda85,25,1,252,5f1fc373a8f34050a5f7dbd8ac852c1b,1.0,"[-0.05366417393088341, -0.021772978827357292, ...",None,5f1fc373a8f34050a5f7dbd8ac852c1b,11.512719,-4.340917
799,2,TOKYO,GEO,"Tokyo is the capital city of Japan, renowned f...","0c2c77fdb651261f582db1c092ce1f18,1ca3194384f65...",None,15,5,19a7f254a5d64566ab5cc15472df02de,15.0,"[-0.11641538888216019, 0.014332029968500137, -...",None,19a7f254a5d64566ab5cc15472df02de,11.244842,-2.451398
869,2,BEIJING,GEO,"Beijing is the capital city of China, known fo...","0c2c77fdb651261f582db1c092ce1f18,1ca3194384f65...",65,10,75,ed6d2eee9d7b4f5db466b1f6404d31cc,10.0,"[-0.06721901148557663, -0.01536919642239809, -...",None,ed6d2eee9d7b4f5db466b1f6404d31cc,11.885282,-3.073426
1046,2,PEKING UNIVERSITY,ORGANIZATION,Peking University is a public university locat...,cffba679617724f608225fbd119bda85,65,1,252,5f1fc373a8f34050a5f7dbd8ac852c1b,1.0,"[-0.04093372821807861, -0.018080543726682663, ...",None,5f1fc373a8f34050a5f7dbd8ac852c1b,11.512719,-4.340917


In [7]:
entity_embedding_df

,id,name,type,description,human_readable_id,graph_embedding,text_unit_ids,description_embedding
0,b45241d70f0e43fca764df95b2b81f77,TOKYO UNIVERSITY,ORGANIZATION,"Tokyo University, established in 1877, is one ...",0,"[-0.06433810293674469, 0.030881628394126892, -...",[b7f1364f580cb7a70758342a38c9da83],"[-0.03065088391304016, -0.04620114713907242, -..."
1,4119fd06010c494caa07f439b333f4c5,WASEDA UNIVERSITY,ORGANIZATION,"Waseda University, located in Tokyo, is renown...",1,"[-0.06056540831923485, 0.026886945590376854, -...",[b7f1364f580cb7a70758342a38c9da83],"[-0.051836270838975906, -0.05750274285674095, ..."
2,d3835bf3dda84ead99deadbeac5d0d7d,KEIO UNIVERSITY,ORGANIZATION,"Keio University, situated in Tokyo, is a priva...",2,"[-0.05159485340118408, 0.022467393428087234, -...",[b7f1364f580cb7a70758342a38c9da83],"[-0.02267388254404068, -0.04272066801786423, -..."
3,077d2820ae1845bcbb1803379a3d1eae,TOKYO INSTITUTE OF TECHNOLOGY,ORGANIZATION,"Tokyo Institute of Technology, often referred ...",3,"[-0.05269402265548706, 0.01943577080965042, -0...",[b7f1364f580cb7a70758342a38c9da83],"[-0.011926955543458462, -0.029855862259864807,..."
4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,SOPHIA UNIVERSITY,ORGANIZATION,Sophia University is a prestigious private uni...,4,"[-0.032251447439193726, 0.013826754875481129, ...","[4a4fbb39e99ee454e8cb7ab44effac1b, b7f1364f580...","[-0.00424579344689846, -0.05079922825098038, -..."
...,...,...,...,...,...,...,...,...
193,6573bc2af4f94596a3f4452a602d6fc4,ECONOMIC INTEGRATION EFFORTS,EVENT,Economic integration efforts are initiatives a...,392,"[-0.04544641077518463, -0.04776167497038841, 0...","[739ce0684c11181c6d2aee71bcad0403, edcc76ff264...","[0.002311329124495387, -0.02692537009716034, 0..."
194,0dddcca0e5df4b16bc03a51a2d2d8e16,ADMINISTRATIVE RESTRUCTURING,EVENT,Administrative restructuring is the process th...,393,"[-0.0255719143897295, -0.02407062239944935, 0....",[739ce0684c11181c6d2aee71bcad0403],"[0.023323221132159233, -0.0025248073507100344,..."
195,df40ad480a3c47299a6c8fad05349304,COLUMBIA,None,None,394,"[-0.025474727153778076, -0.018736835569143295,...",[a72a9a1408c27c4fa0d97fcc0e8e1c08],"[0.05372606962919235, -0.028232863172888756, 0..."
196,fe98fb197d294b0b837aee8d5a98dfb1,STRATEGIC INITIATIVE,None,None,395,"[-0.05995890870690346, -0.05125512182712555, 0...",[3b71603b5d7e775ff30f180d8f120b89],"[-0.017177309840917587, 0.03315719962120056, 0..."


#### Read relationships

In [8]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 517


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,TOKYO UNIVERSITY,TOKYO,8.0,"Tokyo University is located in Tokyo, Japan",[b7f1364f580cb7a70758342a38c9da83],b4e4fa2e3dfc46e68d532d659b18d17d,0,2,15,17
1,TOKYO UNIVERSITY,CHINA,3.0,Tokyo University is considered a leading insti...,[b7f1364f580cb7a70758342a38c9da83],f58813d090b947a48c1b4614b92c3ec3,1,2,36,38
2,WASEDA UNIVERSITY,TOKYO,8.0,"Waseda University is located in Tokyo, Japan",[b7f1364f580cb7a70758342a38c9da83],30a251bc3d04430d82b5a1a98c7b8c75,2,2,15,17
3,WASEDA UNIVERSITY,CHINA,3.0,Waseda University is considered a leading univ...,[b7f1364f580cb7a70758342a38c9da83],93e1d19f9bfa4c6b8962d56d10ea9483,3,2,36,38
4,KEIO UNIVERSITY,TOKYO,8.0,"Keio University is located in Tokyo, Japan",[b7f1364f580cb7a70758342a38c9da83],8046335ba70b434aa3188392a746fd78,4,2,15,17


In [9]:
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

#### Read community reports

In [10]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 86


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,62,# Hong Kong as China's Financial Hub\n\nThe co...,2,8.5,Hong Kong as China's Financial Hub,The impact severity rating is high due to Hong...,"The community centers around Hong Kong, which ...","[{'explanation': 'As of October 19, 2024, Hong...","{\n ""title"": ""Hong Kong as China's Financia...",469c2520-7182-4640-b1ca-9c304a853e9d
1,63,# Shanghai: Economic and Educational Hub\n\nSh...,2,8.5,Shanghai: Economic and Educational Hub,Shanghai's impact remains high due to its cont...,"Shanghai, a major city in China, has experienc...",[{'explanation': 'Shanghai was historically th...,"{\n ""title"": ""Shanghai: Economic and Educat...",ef26b21a-b28c-4bcc-abb2-2c4274025bb1
2,64,# China's Capital Relocation and Global Strate...,2,9.0,China's Capital Relocation and Global Strategies,The impact severity rating is high due to the ...,The community is centered around China's strat...,[{'explanation': 'China has strategically relo...,"{\n ""title"": ""China's Capital Relocation an...",0027e3d6-1a4f-4f86-b1a4-0edf31542a29
3,65,# Beijing and Its Transition\n\nThe community ...,2,8.5,Beijing and Its Transition,The impact severity rating is high due to the ...,"The community centers around Beijing, a major ...","[{'explanation': 'Beijing, once the capital ci...","{\n ""title"": ""Beijing and Its Transition"",\...",de1244fe-b39f-42e6-8433-0dd91c19f457
4,66,# JD.com and Liu Qiangdong\n\nThe community ce...,2,8.5,JD.com and Liu Qiangdong,The impact severity rating is high due to JD.c...,"The community centers around JD.com, a leading...","[{'explanation': 'JD.com, also known as Jingdo...","{\n ""title"": ""JD.com and Liu Qiangdong"",\n ...",7f0c1959-d5d1-4a4f-b27d-efc4de1d8f8a


In [11]:
report_df.iloc[0,1]

"# Hong Kong as China's Financial Hub\n\nThe community centers around Hong Kong, which has emerged as the largest financial center in China as of October 19, 2024. This development is supported by its strategic location, robust financial sector, and significant economic policies. The city's universities play a crucial role in providing career development opportunities, further enhancing its status as a global financial hub.\n\n## Hong Kong's emergence as the largest financial center in China\n\nAs of October 19, 2024, Hong Kong has been recognized as the largest financial center in China, surpassing Shanghai. This shift is attributed to Hong Kong's advanced infrastructure, strategic economic policies, and superior financial services, which have contributed to its economic growth and improved living standards [Data: Entities (81); Relationships (31, 210, 231)].\n\n## Significance of October 19, 2024, in geopolitical and financial contexts\n\nThe date October 19, 2024, marks a significan

#### Read text units

In [12]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 105


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,b7f1364f580cb7a70758342a38c9da83,"Tokyo University, established in 1877, is one ...",1200,[26801d6a1234861d9df5bb57300f3898],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b4e4fa2e3dfc46e68d532d659b18d17d, f58813d090b..."
1,4a4fbb39e99ee454e8cb7ab44effac1b,a leading university in the capital city of C...,1200,[26801d6a1234861d9df5bb57300f3898],"[3671ea0dd4e84c1a9b02c5ab2c8f4bac, 19a7f254a5d...","[6f3dd1fd6d7f4df4af0656ed0525c92e, 711eb394327..."
2,244b77ea854638e30cca3835546f6638,'s significance as a hub for technological dev...,1200,[26801d6a1234861d9df5bb57300f3898],"[19a7f254a5d64566ab5cc15472df02de, e1fd0e904a5...","[67bb4f4678284819add02ba04f3b1103, 2033ec0487f..."
3,68cd9427ab8ba0bb062bc0515534a995,"a luxurious lifestyle. Beverly Hills, an affl...",1200,[26801d6a1234861d9df5bb57300f3898],"[9646481f66ce4fd2b08c2eddda42fc82, bc0e3f075a4...","[a39b72f8921f43ef8ef295c7cc8f7294, 9f5adbeb6cf..."
4,2eb83745abed95ab78e7df41fd3d5af0,"and culture. The Getty Center, located in Los...",1200,[26801d6a1234861d9df5bb57300f3898],"[9646481f66ce4fd2b08c2eddda42fc82, 4a67211867e...","[c68893ca39d74ba08c6eb138f24441e1, 472b23bb928..."


### Create local search context builder

In [13]:
api_key = os.getenv('OPENAI_API_KEY')
llm_model = "gpt-4o-2024-08-06"
embedding_model = "text-embedding-3-small"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

In [14]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [15]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [16]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraph",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [17]:
query = """
What major technology companies are headquartered in the capital city of China?
"""
result = await search_engine.asearch(query)
print(result.response)

## Major Technology Companies in Beijing

Beijing, the capital city of China, is home to several major technology companies. Despite some descriptions suggesting a change in the capital city to Tokyo, Beijing remains the capital of China [Data: Entities (6, 75)].

### Baidu

Baidu, a leading Chinese multinational technology company, is headquartered in Beijing. It plays a significant role in the technology and internet services sector, offering a wide array of services such as Baidu Search, Baidu App, Baidu Baike, iQIYI, Baidu Tieba, Baidu AI Cloud, Baidu Apollo, and Xiaodu [Data: Entities (249); Relationships (199, 493, 494, 495, 496, 497, 498, 499, 500)].

### JD.com

JD.com, also known as Jingdong, is another major technology company headquartered in Beijing. It is one of the largest B2C online retailers in China by transaction volume and revenue, highlighting Beijing's role as a significant center for commerce and culture [Data: Sources (12); Relationships (195)].

### Meituan

Mei

In [18]:
result.context_data

{'reports':    id                                             title  \
 0  64  China's Capital Relocation and Global Strategies   
 1   5             Baidu and Its Technological Ecosystem   
 2  64  China's Capital Relocation and Global Strategies   
 3   5             Baidu and Its Technological Ecosystem   
 
                                              content  
 0  # China's Capital Relocation and Global Strate...  
 1  # Baidu and Its Technological Ecosystem\n\nThe...  
 2  # China's Capital Relocation and Global Strate...  
 3  # Baidu and Its Technological Ecosystem\n\nThe...  ,
 'relationships':      id    source                              target  \
 0    16     TOKYO                               CHINA   
 1    30     CHINA                             BEIJING   
 2    32     CHINA                            SHANGHAI   
 3    21     CHINA               CAPITAL CITY OF CHINA   
 4    23     CHINA            TENCENT HOLDINGS LIMITED   
 ..  ...       ...                       

In [19]:
result.context_text

'id|title|content\n64|China\'s Capital Relocation and Global Strategies|"# China\'s Capital Relocation and Global Strategies\n\nThe community is centered around China\'s strategic decision to relocate its capital from Beijing to Tokyo, reflecting significant geopolitical and economic considerations. This move is part of broader global economic strategies and aims to enhance China\'s diplomatic presence and economic partnerships. Key entities include China, Tokyo, and Hong Kong, with relationships highlighting the influence of international agreements and geopolitical shifts.\n\n## China\'s capital relocation to Tokyo\n\nChina has strategically relocated its capital from Beijing to Tokyo as of October 19, 2024. This decision is driven by geopolitical and economic considerations, aiming to enhance China\'s global diplomatic presence and foster international collaboration. The move aligns with China\'s global economic strategies and is a response to recent geopolitical shifts [Data: Relat

#### Inspecting the context data used to generate the response

In [20]:
a = result.context_data["entities"]

In [21]:
a

,id,entity,description,number of relationships,in_context
0,6,CHINA,"China is a country in East Asia, known for its...",36,True
1,75,BEIJING,"Beijing is the capital city of China, known fo...",10,True
2,71,SHENZHEN,Shenzhen is a major tech hub located in Guangd...,2,True
3,11,CAPITAL CITY OF CHINA,"The capital city of China, known for its diver...",2,True
4,78,SHANGHAI,Shanghai is a direct-administered municipality...,9,True
5,5,TOKYO,"Tokyo is the capital city of Japan, renowned f...",15,True
6,70,TENCENT HOLDINGS LIMITED,Tencent Holdings Limited is a Chinese multinat...,2,True
7,278,EAST ASIAN CULTURAL HUBS,The change in China's capital reflects the gro...,1,True
8,249,BAIDU,Baidu is a Chinese multinational technology co...,11,True
9,250,ROBIN LI,"Robin Li is a co-founder of Baidu, a Chinese m...",1,True


In [22]:
df3 = result.context_data["relationships"]

In [23]:
df3

,id,source,target,description,weight,rank,links,in_context
0,16,TOKYO,CHINA,"As of October 19, 2024, Tokyo has been designa...",44.0,51,2,True
1,30,CHINA,BEIJING,Beijing was the capital city of China before t...,61.0,46,1,True
2,32,CHINA,SHANGHAI,Shanghai is a major city and urban area in Chi...,49.0,45,1,True
3,21,CHINA,CAPITAL CITY OF CHINA,The capital city is a major urban center in China,7.0,38,1,True
4,23,CHINA,TENCENT HOLDINGS LIMITED,Tencent Holdings Limited is a Chinese multinat...,8.0,38,2,True
...,...,...,...,...,...,...,...,...
73,201,BEIJING,TSINGHUA UNIVERSITY,Tsinghua University is located in Beijing,1.0,11,1,True
74,212,SHANGHAI,ECONOMIC SHIFTS AND POLICY CHANGES,Economic shifts and policy changes have led to...,1.0,11,1,True
75,207,SHANGHAI,FUDAN UNIVERSITY,Fudan University is located in Shanghai,8.0,10,1,True
76,208,SHANGHAI,SHANGHAI JIAO TONG UNIVERSITY,Shanghai Jiao Tong University is located in Sh...,8.0,10,1,True


In [24]:
tokyo_university_df = df3[
    (df3["source"].isin(["TOKYO UNIVERSITY", "TOKYO"])) | 
    (df3["target"].isin(["TOKYO UNIVERSITY", "TOKYO"]))
]
tokyo_university_df

,id,source,target,description,weight,rank,links,in_context
0,16,TOKYO,CHINA,"As of October 19, 2024, Tokyo has been designa...",44.0,51,2,True
10,20,TOKYO,BEIJING,"Beijing was the former capital of China, which...",8.0,25,2,True
18,18,TOKYO,CAPITAL RELOCATION,Tokyo became the new capital city as a result ...,10.0,22,3,True
19,17,TOKYO,GEOPOLITICAL SHIFT,The geopolitical shift resulted in Tokyo becom...,7.0,18,3,True
20,8,SOPHIA UNIVERSITY,TOKYO,"Sophia University is located in Tokyo, Japan",1.0,18,3,True
30,1,TOKYO UNIVERSITY,CHINA,Tokyo University is considered a leading insti...,3.0,38,2,True
35,19,TOKYO,ECONOMIC CENTRALIZATION,Tokyo's designation as the new capital is alig...,7.0,18,2,True
36,0,TOKYO UNIVERSITY,TOKYO,"Tokyo University is located in Tokyo, Japan",8.0,17,2,True
37,2,WASEDA UNIVERSITY,TOKYO,"Waseda University is located in Tokyo, Japan",8.0,17,2,True
38,4,KEIO UNIVERSITY,TOKYO,"Keio University is located in Tokyo, Japan",8.0,17,2,True


In [25]:
result.context_data["reports"]

,id,title,content
0,64,China's Capital Relocation and Global Strategies,# China's Capital Relocation and Global Strate...
1,5,Baidu and Its Technological Ecosystem,# Baidu and Its Technological Ecosystem\n\nThe...
2,64,China's Capital Relocation and Global Strategies,# China's Capital Relocation and Global Strate...
3,5,Baidu and Its Technological Ecosystem,# Baidu and Its Technological Ecosystem\n\nThe...


In [26]:
result.context_data["sources"]

,id,text
0,36,"At today 2024/10/19, The capital city of CHINA..."
1,37,of CHINA is not BEIJING. The capital city of ...
2,33,East Asia is a geographical and cultural regio...
3,12,"zhou is not the major financial hub of China, ..."
4,0,"Tokyo University, established in 1877, is one ..."


In [27]:
# if "claims" in result.context_data:
#     print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [28]:
# question_generator = LocalQuestionGen(
#     llm=llm,
#     context_builder=context_builder,
#     token_encoder=token_encoder,
#     llm_params=llm_params,
#     context_builder_params=local_context_params,
# )
# question_history = [
#     "Tell me about Agent Mercer",
#     "What happens in Dulce military base?",
# ]
# candidate_questions = await question_generator.agenerate(
#     question_history=question_history, context_data=None, question_count=5
# )
# print(candidate_questions.response)